In [6]:
from memory_profiler import profile
import pennylane as qml
import pennylane.numpy as np
import time
from functools import wraps
from line_profiler import LineProfiler
import csv

In [9]:
# wires_m=[0]
# wires_solution=[1,2,3,4,5,6,7,8,9,10,11]

wires_m=[0,1]
wires_solution=[2,3,4,5,6,7,8,9,10,11,12]

# wires_m=[0,1,2]
# wires_solution=[3,4,5,6,7,8,9,10,11,12,13]
#
# wires_m=[0,1,2,3]
# wires_solution=[4,5,6,7,8,9,10,11,12,13,14]

# wires_m=[0,1,2,3,4]
# wires_solution=[5,6,7,8,9,10,11,12,13,14,15]
#
# add_time=0
# mul_time=0
# total_time=0

In [10]:
dev = qml.device("default.qubit", wires=wires_m  + wires_solution, shots=1)
# dev = qml.device("qiskit.ibmq", wires=wires_m + wires_solution, shots=1000, backend='',overwrite=True, ibmqx_token='')

n_wires = len(dev.wires)

In [ ]:
# def calculate_time_add(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         start = time.time()
#         result = func(*args, **kwargs)
#         end = time.time()
#         execution_time = (end - start) * 1000  # 转换成毫秒
#         global add_time
#         add_time+=execution_time
#         return result
#     return wrapper
#
# def calculate_time_mul(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         start = time.time()
#         result = func(*args, **kwargs)
#         end = time.time()
#         execution_time = (end - start) * 1000  # 转换成毫秒
#         global mul_time
#         mul_time+=execution_time
#         return result
#     return wrapper
#
# def calculate_time(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         start = time.time()
#         result = func(*args, **kwargs)
#         end = time.time()
#         execution_time = (end - start) * 1000  # 转换成毫秒
#         global total_time
#         total_time=execution_time
#         return result
#     return wrapper

In [11]:
def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

In [12]:
def multiplication(k,wires_m, wires_solution):
    for i in range(len(wires_m)-1,-1,-1):
        qml.ctrl(add_k_fourier, control=wires_m[i])(k, wires_solution[len(wires_m)-1-i:])


# @calculate_time_mul
@qml.qnode(dev)
def mul(m,k):
    qml.BasisEmbedding(m, wires=wires_m)  # m encoding
    qml.QFT(wires=wires_solution)
    multiplication(k,wires_m, wires_solution)
    qml.adjoint(qml.QFT)(wires=wires_solution)
    return qml.sample(wires=wires_solution)

In [13]:
# @calculate_time_add
@qml.qnode(dev)
def sum3(m, k):
    qml.BasisEmbedding(m, wires=range(len(wires_solution)))  # m encoding
    qml.QFT(wires=range(len(wires_solution)))  # step 1
    add_k_fourier(k, range(len(wires_solution)))  # step 2
    qml.adjoint(qml.QFT)(wires=range(len(wires_solution)))  # step 3
    return qml.sample(wires=range(len(wires_solution)))

In [14]:
k=2
val=2
matrix_a=np.full((k,k),val)
matrix_b=np.full((k,k),val)

n=matrix_a.shape[0]
m=matrix_b.shape[1]
matrix_c=np.zeros((n,m))

In [16]:
def dot(a,b):
    listx=[]
    x=0
    for i in range(a.shape[0]):
        mulnum=mul(int(a[i]),int(b[i]))
        mulnum=int(''.join(map(str, list(mulnum))), 2)
        listx.append(mulnum)
    i=0
    while(listx.__len__()>1):
        i=0
        sumnum=sum3(listx[i], listx[i+1])
        sumnum=int(''.join(map(str, list(sumnum))), 2)
        x=x+sumnum
        i=i+1
        if listx.__len__()>=2:
            listx=listx[2:]
            listx.insert(0,sumnum)
    return listx[0]

# @calculate_time
def matrix_multiplication_mulpro():
    for i in range(n):
        for j in range(m):
            matrix_c[i, j] = dot(matrix_a[i], matrix_b[:, j])
            # print(matrix_c[i,j])
            print(matrix_c)

In [17]:
if __name__=='__main__':
    matrix_multiplication_mulpro()

    # csv_file = open('multication_mulpro.csv', 'a', newline='')
    # writer = csv.writer(csv_file)
    # data = [['size', 'value', 'add_time', 'mul_time', 'total_time'], [k, val, add_time, mul_time, total_time]]
    # writer.writerows(data)
    # csv_file.close()

[[8. 0.]
 [0. 0.]]
[[8. 8.]
 [0. 0.]]
[[8. 8.]
 [8. 0.]]
[[8. 8.]
 [8. 8.]]
